In [78]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize
import json
from src.usamap import *
from src.nba import country_with_nba_team
from src.transformToGeoPoint import transformToGeoPoint
from src.findwyl import *
from src.wtematch import*


In [2]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

Vamos a utilizar la base de datos de companies, a traves de pymongo.
En primer lugar vamos a observar los diferentes tipos de empresas que existen en nuestra base da datos.

In [3]:
cursor = db.list_collections()
list(cursor)

[{'name': 'offices',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('5b475f7a-d4d3-4662-a35a-d6259dbe9b24')},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'companies.offices'}},
 {'name': 'companies',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('baa69a23-5c05-4e83-9e9f-c60919550dac')},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'companies.companies'}}]

In [4]:
print(db.companies.distinct("category_code"))

[None, 'advertising', 'analytics', 'automotive', 'biotech', 'cleantech', 'consulting', 'design', 'ecommerce', 'education', 'enterprise', 'fashion', 'finance', 'games_video', 'government', 'hardware', 'health', 'hospitality', 'legal', 'local', 'manufacturing', 'medical', 'messaging', 'mobile', 'music', 'nanotech', 'network_hosting', 'news', 'nonprofit', 'other', 'photo_video', 'public_relations', 'real_estate', 'search', 'security', 'semiconductor', 'social', 'software', 'sports', 'transportation', 'travel', 'web']


- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

- Developers like to be near successful tech startups that have raised at least 1 Million dollars.


In [5]:
companies_around=['design','fashion','games_video','mobile','software','web']

In [6]:
maybefriends=list(db.companies.find({"$and": [ {"category_code":{"$in":companies_around}},{"funding_rounds.raised_amount":{"$gte":1000000}}] },{"offices":1,"name":1,"category_code":1}))

In [7]:
data=pd.DataFrame(maybefriends)

In [8]:
offices = data.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")

In [9]:
offices

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,,,,,San Francisco,CA,USA,37.775196,-122.419204
2,,,,,New York City,NY,USA,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1784,HQ,,,94041,Mountain View,CA,USA,NaN,NaN
1785,World Headquarters,36 rue Vivienne,,75002,PARIS,None,FRA,48.869995,2.340695
1786,Headquarters,"2030 First Avenue, Ste 300",,98121,Seattle,WA,USA,47.611331,-122.343058
1787,Corporate Address,4514 Chamblee Dunwoody Rd.,Suite 188,30338,Dunwoody,GA,USA,37.590056,-122.347654


In [10]:
clean_offices = pd.concat([data[["name","_id","category_code"]], offices], axis=1)
clean_offices = clean_offices.rename(columns={"_id":"company_id"})


Transformamos lat y long en una nueva columna en formato point

In [11]:
clean_offices["geopoint"] = clean_offices.apply(transformToGeoPoint, axis=1)

Solo buscamos en USA

In [12]:
data_usa=clean_offices.loc[clean_offices["country_code"]=="USA"]

Borramos aquellas filas que contengan nulos en lat and long

In [13]:
data_usa=data_usa.dropna(subset=['latitude', 'longitude'])

In [14]:
#Casting a str company column para poderlo exportar a json
data_usa["company_id"] = data_usa["company_id"].apply(lambda e: str(e))
data_usa.to_json("offices.json",orient="records")

In [15]:
#vamos a importar a jupyter nuestro nuevo ya con geopoint index 2sphere , sera necesario para mas adelante
!mongoimport -d companies -c offices --jsonArray offices.json

2020-06-30T00:13:38.364+0200	connected to: mongodb://localhost/
2020-06-30T00:13:38.472+0200	1175 document(s) imported successfully. 0 document(s) failed to import.


Antes de continuar vamos a representar un mapa de todas las compañias las cuales, ya cumplen dos requisitos, mencionado anteriormete:

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

- Developers like to be near successful tech startups that have raised at least 1 Million dollars.


Para ello generaremos un mapa focused en USA , en el cual represente todas las compañias segun su sector

In [16]:

m=folium.Map(location=[41,-101],zoom_start=4)
dict_colors={"design":"red","fashion":"blue","games_video":"green","mobile":"gray","software":"orange","web":"black"}
usamap(m,data_usa,dict_colors)

In [17]:
m

Como podemos observar , la mayor distribucion de puntos o de agromeracion de compañias en este caso se distribuyen entre West Coast e East Coast. De todos modos para cerciorarnos, vamos a obtener el numero por estados, y por ciudades.

In [18]:
print(data_usa["state_code"].value_counts()[:3])
print(data_usa["city"].value_counts()[:3])

CA    613
NY    119
MA     91
Name: state_code, dtype: int64
San Francisco    157
New York         104
Mountain View     45
Name: city, dtype: int64


En nuestro caso a pesar de que CA ocupa mucho mayor porcentaje de compañias , nos quedaremos con NY que cuenta con una red empresarial que opera a nivel mundial y ciertas ventajas fiscales podrían hacer de Nueva York el nuevo y principal centro neurálgico de startups: Sillicon Alley,

In [19]:
citys_opction='New York'

Segun los datos, vamos a seguir la busqueda basandono en NY como centro de nuestra posible oficina, vamos a comprobar que la ciudad cuenta con equipo y estadio de basket.


In [20]:
if citys_opction in country_with_nba_team :
        print(True)

True


In [21]:
data_usa.head()

,name,company_id,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geopoint
0,Wetpaint,52cdef7c4bab8bd675297d8a,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,Wetpaint,52cdef7c4bab8bd675297d8a,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
1,Geni,52cdef7c4bab8bd675297d91,web,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,StumbleUpon,52cdef7c4bab8bd675297d95,web,,,,,San Francisco,CA,USA,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,..."
3,Gizmoz,52cdef7c4bab8bd675297d96,web,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."


Vamos a filtrar nuestro data, para solo contar con companies de NY.

In [75]:
data_NY=data_usa.loc[data_usa["state_code"]=="NY"]
data_NY

,name,company_id,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geopoint
0,Wetpaint,52cdef7c4bab8bd675297d8a,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
5,Joost,52cdef7c4bab8bd675297d9d,games_video,,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447,"{'type': 'Point', 'coordinates': [-74.0094471,..."
15,Meetup,52cdef7c4bab8bd675297db2,web,None,632 Broadway,10th Floor,10012,New York,NY,USA,40.726040,-73.995722,"{'type': 'Point', 'coordinates': [-73.995722, ..."
17,Jingle Networks,52cdef7c4bab8bd675297daf,mobile,,475 Park Ave South,10th Floor,10016,New York,NY,USA,37.480999,-122.173887,"{'type': 'Point', 'coordinates': [-122.173887,..."
24,Livestream,52cdef7c4bab8bd675297dc0,games_video,Livestream HQ,"111 8th Avenue, #1509",,10011,New York,NY,USA,40.726155,-73.995625,"{'type': 'Point', 'coordinates': [-73.995625, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668,Exent,52cdef7f4bab8bd67529c1a9,games_video,Sales & Marketing,632 W. 28th St.,8th Floor,10001,New York,NY,USA,40.752380,-74.005568,"{'type': 'Point', 'coordinates': [-74.005568, ..."
1689,WiNetworks,52cdef7f4bab8bd67529c2d1,mobile,HQ,"555 West 57th Street, Suite 1326",,10019,New York,NY,USA,40.770437,-73.990426,"{'type': 'Point', 'coordinates': [-73.990426, ..."
1706,ChallengePost,52cdef7f4bab8bd67529c343,web,,425 West 13th Street,Suite 504,10014,New York,NY,USA,40.740804,-74.007170,"{'type': 'Point', 'coordinates': [-74.00717, 4..."
1717,StyleCaster,52cdef7f4bab8bd67529c3bc,web,,49 W 27th St Uppr A,,10001-6936,New York City,NY,USA,40.745050,-73.990098,"{'type': 'Point', 'coordinates': [-73.9900984,..."


Vamos a basarnos en las coordenadadas centricuas de las ciudades basandonos en los mapas para sacar todos lo starbucks:
    -New York   /  Manhattan       [40.7834282,-73.9662476]

In [23]:
center_NY=[40.7834282,-73.9662476]

- Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
- Account managers need to travel a lot(Airport)
- The CEO is Vegan
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog "Pepe" needs a hairdresser every month. Ensure there's one not too far away.



Must, lista de todo los lugares que deben cumplir el lugar que ocupemos de las lsita de compañias .

In [105]:
must=["Starbucks","airport","vegan food","basketball stadium","dog hairdresser"]

NY=[ findwyl(center_NY,x) for x in must]
NY

{'meta': {'code': 200, 'requestId': '5efa6c66bbdb0a49a67354f8'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': '$-$$$$', 'key': 'price'}, {'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'New York', 'headerFullLocation': 'New York', 'headerLocationGranularity': 'city', 'query': 'starbucks', 'totalResults': 108, 'suggestedBounds': {'ne': {'lat': 40.87342829000009, 'lng': -73.84760794593946}, 'sw': {'lat': 40.693428109999914, 'lng': -74.08488725406055}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '57eeb8e3498e1dd086ebf03f', 'name': 'Starbucks Reserve', 'location': {'address': '1142 Madison Ave', 'crossStreet': '84th & 85th St', 'lat': 40.77984961470853, 'lng': -73.95958442027471, 'labeledLatLngs': [{'label': 'display', 'lat': 40.77984961470853, 'lng': -73.95958442

{'meta': {'code': 200, 'requestId': '5efa6befa7f69f5c742780fc'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'New York', 'headerFullLocation': 'New York', 'headerLocationGranularity': 'city', 'query': 'airport', 'totalResults': 193, 'suggestedBounds': {'ne': {'lat': 40.87342829000009, 'lng': -73.84760794593946}, 'sw': {'lat': 40.693428109999914, 'lng': -74.08488725406055}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ace6c89f964a52078d020e3', 'name': 'LaGuardia Airport (LGA) (LaGuardia Airport)', 'location': {'address': 'Grand Central Pkwy', 'lat': 40.77288813003166, 'lng': -73.86880874633789, 'labeledLatLngs': [{'label': 'display', 'lat': 40.77288813003166, 'lng': -73.86880874633789}], 'distance': 8297, 'postalCode': '

{'meta': {'code': 200, 'requestId': '5efa6d01587699312d651638'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}, {'name': '$-$$$$', 'key': 'price'}]}, 'headerLocation': 'New York', 'headerFullLocation': 'New York', 'headerLocationGranularity': 'city', 'query': 'vegan food', 'totalResults': 206, 'suggestedBounds': {'ne': {'lat': 40.87342829000009, 'lng': -73.84760794593946}, 'sw': {'lat': 40.693428109999914, 'lng': -74.08488725406055}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4a6c86acf964a520b7d01fe3', 'name': 'Peacefood Cafe', 'location': {'address': '460 Amsterdam Ave', 'crossStreet': '82nd St', 'lat': 40.78516588233606, 'lng': -73.97706383152412, 'labeledLatLngs': [{'label': 'display', 'lat': 40.78516588233606, 'lng': -73.97706383152412},

{'meta': {'code': 200, 'requestId': '5efa6c5a7f806936912feefd'}, 'response': {'warning': {'text': 'There aren\'t a lot of results for "basketball stadium." Try something more general, reset your filters, or expand the search area.'}, 'headerLocation': 'New York', 'headerFullLocation': 'New York', 'headerLocationGranularity': 'city', 'query': 'basketball stadium', 'totalResults': 2, 'suggestedBounds': {'ne': {'lat': 40.87342829000009, 'lng': -73.84760794593946}, 'sw': {'lat': 40.693428109999914, 'lng': -74.08488725406055}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ae6363ef964a520aba521e3', 'name': 'Madison Square Garden', 'location': {'address': '4 Penn Plz', 'crossStreet': 'btwn 7th & 8th Ave', 'lat': 40.75075196505169, 'lng': -73.99354219436646, 'labeledLatLngs': [{'label': 'display', 'lat': 40.7507519

[[{'Place': 'Starbucks',
   'address': '1142 Madison Ave (84th & 85th St) New York, NY 10028 United States',
   'distance': 0.688,
   'location': {'address': '1142 Madison Ave',
    'crossStreet': '84th & 85th St',
    'lat': 40.77984961470853,
    'lng': -73.95958442027471,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.77984961470853,
      'lng': -73.95958442027471}],
    'distance': 688,
    'postalCode': '10028',
    'cc': 'US',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['1142 Madison Ave (84th & 85th St)',
     'New York, NY 10028',
     'United States']},
   'latitude': 40.77984961470853,
   'longitude': -73.95958442027471},
  {'Place': 'Starbucks',
   'address': '805 Columbus Ave (at W 100th St) New York, NY 10025 United States',
   'distance': 1.33,
   'location': {'address': '805 Columbus Ave',
    'crossStreet': 'at W 100th St',
    'lat': 40.79536902,
    'lng': -73.96558901,
    'labeledLatLngs': [{'labe

## Starbucks

In [106]:
datany=pd.DataFrame(NY[0])
datany

,Place,address,distance,location,latitude,longitude
0,Starbucks,"1142 Madison Ave (84th & 85th St) New York, NY...",0.688,"{'address': '1142 Madison Ave', 'crossStreet':...",40.779850,-73.959584
1,Starbucks,"805 Columbus Ave (at W 100th St) New York, NY ...",1.330,"{'address': '805 Columbus Ave', 'crossStreet':...",40.795369,-73.965589
2,Starbucks,"245 E 93rd St (at 2nd Ave) New York, NY 10128 ...",1.525,"{'address': '245 E 93rd St', 'crossStreet': 'a...",40.782715,-73.948169
3,Starbucks,"7301 River Rd. North Bergen, NJ 07047 United S...",2.906,"{'address': '7301 River Rd.', 'lat': 40.792725...",40.792725,-73.998470
4,Starbucks,55-1 Riverwalk Pl (at Port Imperial Blvd) West...,3.432,"{'address': '55-1 Riverwalk Pl', 'crossStreet'...",40.783092,-74.006968
5,Starbucks,"New York, NY 10036 United States",3.120,"{'lat': 40.759393, 'lng': -73.985288, 'labeled...",40.759393,-73.985288
6,Starbucks,"600 Madison Ave (at E 58th St) New York, NY 10...",2.321,"{'address': '600 Madison Ave', 'crossStreet': ...",40.763077,-73.972284
7,Starbucks,"1542 Third Ave (at E 87th) New York, NY 10028 ...",1.154,"{'address': '1542 Third Ave', 'crossStreet': '...",40.779546,-73.953552
8,Starbucks,"475 West 57th st (at 10th Ave) New York, NY 10...",2.434,"{'address': '475 West 57th st', 'crossStreet':...",40.769371,-73.988368
9,Starbucks,"1515 York Ave (at E 80th St) New York, NY 1002...",1.843,"{'address': '1515 York Ave', 'crossStreet': 'a...",40.772356,-73.949984


In [107]:
datany
datany["geopoint"] = datany.apply(transformToGeoPoint, axis=1)
datany

,Place,address,distance,location,latitude,longitude,geopoint
0,Starbucks,"1142 Madison Ave (84th & 85th St) New York, NY...",0.688,"{'address': '1142 Madison Ave', 'crossStreet':...",40.779850,-73.959584,"{'type': 'Point', 'coordinates': [-73.95958442..."
1,Starbucks,"805 Columbus Ave (at W 100th St) New York, NY ...",1.330,"{'address': '805 Columbus Ave', 'crossStreet':...",40.795369,-73.965589,"{'type': 'Point', 'coordinates': [-73.96558901..."
2,Starbucks,"245 E 93rd St (at 2nd Ave) New York, NY 10128 ...",1.525,"{'address': '245 E 93rd St', 'crossStreet': 'a...",40.782715,-73.948169,"{'type': 'Point', 'coordinates': [-73.94816913..."
3,Starbucks,"7301 River Rd. North Bergen, NJ 07047 United S...",2.906,"{'address': '7301 River Rd.', 'lat': 40.792725...",40.792725,-73.998470,"{'type': 'Point', 'coordinates': [-73.99847, 4..."
4,Starbucks,55-1 Riverwalk Pl (at Port Imperial Blvd) West...,3.432,"{'address': '55-1 Riverwalk Pl', 'crossStreet'...",40.783092,-74.006968,"{'type': 'Point', 'coordinates': [-74.00696827..."
5,Starbucks,"New York, NY 10036 United States",3.120,"{'lat': 40.759393, 'lng': -73.985288, 'labeled...",40.759393,-73.985288,"{'type': 'Point', 'coordinates': [-73.985288, ..."
6,Starbucks,"600 Madison Ave (at E 58th St) New York, NY 10...",2.321,"{'address': '600 Madison Ave', 'crossStreet': ...",40.763077,-73.972284,"{'type': 'Point', 'coordinates': [-73.97228402..."
7,Starbucks,"1542 Third Ave (at E 87th) New York, NY 10028 ...",1.154,"{'address': '1542 Third Ave', 'crossStreet': '...",40.779546,-73.953552,"{'type': 'Point', 'coordinates': [-73.95355245..."
8,Starbucks,"475 West 57th st (at 10th Ave) New York, NY 10...",2.434,"{'address': '475 West 57th st', 'crossStreet':...",40.769371,-73.988368,"{'type': 'Point', 'coordinates': [-73.98836833..."
9,Starbucks,"1515 York Ave (at E 80th St) New York, NY 1002...",1.843,"{'address': '1515 York Ave', 'crossStreet': 'a...",40.772356,-73.949984,"{'type': 'Point', 'coordinates': [-73.949984, ..."


In [108]:
def wtematch(datany):
    resultados=[]
    for street in datany["geopoint"]:
        try:
            q=db.offices.find(geoQueryNear(street, radius=500),{"_id":0,"company_id":0})
            resultados+=q
        except Exception as e:
            continue
    return resultados

In [109]:
#Empresas que tienen un starbucks menos 1 km
stroffices = pd.DataFrame(wtematch(datany))
stroffices


,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geopoint
0,Tacoda,web,None,None,None,None,New York,NY,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
1,Blog Talk Radio,web,None,,,,New York,NY,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
2,Outbrain,web,New York,116 E. 16th Street,12th Floor,10003,New York City,NY,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
3,Tacoda,web,None,None,None,None,New York,NY,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
4,Blog Talk Radio,web,None,,,,New York,NY,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
70,Firm58,software,New York Office (Sales),,,,New York,NY,USA,40.766734,-73.988265,"{'type': 'Point', 'coordinates': [-73.988265, ..."
71,Firm58,software,New York Office (Sales),,,,New York,NY,USA,40.766734,-73.988265,"{'type': 'Point', 'coordinates': [-73.988265, ..."
72,Firm58,software,New York Office (Sales),,,,New York,NY,USA,40.766734,-73.988265,"{'type': 'Point', 'coordinates': [-73.988265, ..."
73,Firm58,software,New York Office (Sales),,,,New York,NY,USA,40.766734,-73.988265,"{'type': 'Point', 'coordinates': [-73.988265, ..."


In [110]:
stroffices.to_json("stroffices.json",orient="records")

In [111]:
!mongoimport -d offices -c stroffices --jsonArray offices.json

2020-06-30T00:36:33.873+0200	connected to: mongodb://localhost/
2020-06-30T00:36:33.946+0200	1175 document(s) imported successfully. 0 document(s) failed to import.


In [112]:
datany=pd.DataFrame(NY[1])

datany["geopoint"] = datany.apply(transformToGeoPoint, axis=1)
datany

,Place,address,distance,location,latitude,longitude,geopoint
0,airport,"Grand Central Pkwy East Elmhurst, NY 11369 Uni...",8.297,"{'address': 'Grand Central Pkwy', 'lat': 40.77...",40.772888,-73.868809,"{'type': 'Point', 'coordinates': [-73.86880874..."
1,airport,"LaGuardia Airport East Elmhurst, NY 11371 Unit...",6.989,"{'address': 'LaGuardia Airport', 'lat': 40.773...",40.773739,-73.884321,"{'type': 'Point', 'coordinates': [-73.88432092..."
2,airport,"Terminal A (LGA Airport) East Elmhurst, NY 113...",6.869,"{'address': 'Terminal A', 'crossStreet': 'LGA ...",40.773507,-73.885808,"{'type': 'Point', 'coordinates': [-73.88580815..."
3,airport,"Terminal B, 3rd Fl (LaGuardia Airport) East El...",8.007,"{'address': 'Terminal B, 3rd Fl', 'crossStreet...",40.774401,-73.872000,"{'type': 'Point', 'coordinates': [-73.87200014..."
4,airport,"LaGuardia Airport East Elmhurst, NY 11369 Unit...",8.007,"{'address': 'LaGuardia Airport', 'lat': 40.774...",40.774367,-73.872006,"{'type': 'Point', 'coordinates': [-73.87200593..."
5,airport,"LaGuardia Airport East Elmhurst, NY 11369 Unit...",8.917,"{'address': 'LaGuardia Airport', 'lat': 40.769...",40.769236,-73.862135,"{'type': 'Point', 'coordinates': [-73.86213541..."
6,airport,Terminal B (Central Terminal) (LaGuardia Airpo...,7.924,"{'address': 'Terminal B (Central Terminal)', '...",40.776192,-73.872725,"{'type': 'Point', 'coordinates': [-73.87272477..."
7,airport,"LaGuardia Airport East Elmhurst, NY 11369 Unit...",8.572,"{'address': 'LaGuardia Airport', 'lat': 40.771...",40.771502,-73.865780,"{'type': 'Point', 'coordinates': [-73.86577999..."
8,airport,"New York, NY United States",4.431,"{'lat': 40.74404811351841, 'lng': -73.97392544...",40.744048,-73.973925,"{'type': 'Point', 'coordinates': [-73.97392544..."
9,airport,Terminal B (Central Terminal) (LaGuardia Airpo...,7.784,"{'address': 'Terminal B (Central Terminal)', '...",40.775706,-73.874460,"{'type': 'Point', 'coordinates': [-73.87446003..."


In [121]:
def wtematch(datany):
    resultados=[]
    for street in datany["geopoint"]:
        try:
            q=db.offices.find(geoQueryNear(street, radius=500),{"_id":0,"company_id":0})
            resultados+=q
        except Exception as e:
            continue
    return resultados

In [ ]:
str_air_offices.to_json("str_air_offices.json",orient="records")
!mongoimport -d offices -c str_air_offices --jsonArray str_air_offices.json

In [97]:
datany=pd.DataFrame(NY[2])
datany
datany["geopoint"] = datany.apply(transformToGeoPoint, axis=1)
datany

,Place,address,distance,location,latitude,longitude,geopoint
0,vegan food,"460 Amsterdam Ave (82nd St) New York, NY 10024...",0.931,"{'address': '460 Amsterdam Ave', 'crossStreet'...",40.785166,-73.977064,"{'type': 'Point', 'coordinates': [-73.97706383..."
1,vegan food,"1307 3rd Ave (btwn 74th & 75th St.) New York, ...",1.466,"{'address': '1307 3rd Ave', 'crossStreet': 'bt...",40.771407,-73.959138,"{'type': 'Point', 'coordinates': [-73.959138, ..."
2,vegan food,"55 St. Nicholas (West 113th) New York, NY 1002...",2.229,"{'address': '55 St. Nicholas', 'crossStreet': ...",40.800625,-73.952693,"{'type': 'Point', 'coordinates': [-73.952693, ..."
3,vegan food,246 W 48th St (btwn Broadway & 8th Ave) New Yo...,3.037,"{'address': '246 W 48th St', 'crossStreet': 'b...",40.760930,-73.986630,"{'type': 'Point', 'coordinates': [-73.98662972..."
4,vegan food,311 Amsterdam Ave (btwn W 74th St & W 75th St)...,1.224,"{'address': '311 Amsterdam Ave', 'crossStreet'...",40.780298,-73.980179,"{'type': 'Point', 'coordinates': [-73.9801793,..."
5,vegan food,"12 Park Ave (at E 34th St.) New York, NY 10016...",4.186,"{'address': '12 Park Ave', 'crossStreet': 'at ...",40.747549,-73.981124,"{'type': 'Point', 'coordinates': [-73.98112380..."
6,vegan food,"833 Lexington Ave (6th St) New York, NY 10065 ...",2.041,"{'address': '833 Lexington Ave', 'crossStreet'...",40.765090,-73.966052,"{'type': 'Point', 'coordinates': [-73.96605217..."
7,vegan food,"60 W 22nd St (at MacDougal St) New York, NY 10...",5.147,"{'address': '60 W 22nd St', 'crossStreet': 'at...",40.741862,-73.992998,"{'type': 'Point', 'coordinates': [-73.992998, ..."
8,vegan food,"60 W 22nd St (5th and 6th Avenue) New York, NY...",5.133,"{'address': '60 W 22nd St', 'crossStreet': '5t...",40.741972,-73.992919,"{'type': 'Point', 'coordinates': [-73.9929188,..."
9,vegan food,"1321 1st Ave New York, NY 10021 United States",1.980,"{'address': '1321 1st Ave', 'lat': 40.76712827...",40.767128,-73.956846,"{'type': 'Point', 'coordinates': [-73.95684622..."


In [98]:
str_air_vegan_offices = pd.DataFrame(resultados)
str_air_vegan_offices.to_json("str_air_vegan_offices.json",orient="records")
!mongoimport -d offices -c str_air_vegan_offices --jsonArray str_air_vegan_offices.json


NameError: name 'resultados' is not defined

In [99]:
datany=pd.DataFrame(NY[3])
datany
datany["geopoint"] = datany.apply(transformToGeoPoint, axis=1)
datany

,Place,address,distance,location,latitude,longitude,geopoint
0,basketball stadium,"4 Penn Plz (btwn 7th & 8th Ave) New York, NY 1...",4.304,"{'address': '4 Penn Plz', 'crossStreet': 'btwn...",40.750752,-73.993542,"{'type': 'Point', 'coordinates': [-73.99354219..."
1,basketball stadium,"200 E 87th St (btwn 2nd & 3rd Ave) New York, N...",1.190,"{'address': '200 E 87th St', 'crossStreet': 'b...",40.779233,-73.953255,"{'type': 'Point', 'coordinates': [-73.95325541..."


In [100]:
str_air_vegan_nba_offices=pd.DataFrame(wtematch(datany))
str_air_vegan_nba_offices.to_json("str_air_vegan_nba_offices.json",orient="records")
!mongoimport -d offices -c str_air_vegan_nba_offices --jsonArray str_air_vegan_nba_offices.json



2020-06-30T00:33:50.833+0200	connected to: mongodb://localhost/
2020-06-30T00:33:50.836+0200	15 document(s) imported successfully. 0 document(s) failed to import.


In [101]:
datany=pd.DataFrame(NY[4])
datany
datany["geopoint"] = datany.apply(transformToGeoPoint, axis=1)
datany

,Place,address,distance,location,latitude,longitude,geopoint
0,dog hairdresser,"105th St. (@ Riverside Dr.) New York, NY Unite...",2.165,"{'address': '105th St.', 'crossStreet': '@ Riv...",40.802484,-73.971443,"{'type': 'Point', 'coordinates': [-73.97144265..."
1,dog hairdresser,E End Ave and 86th St (East River Promenade) N...,2.161,"{'address': 'E End Ave and 86th St', 'crossStr...",40.773933,-73.943882,"{'type': 'Point', 'coordinates': [-73.94388158..."
2,dog hairdresser,"West 72nd St (Riverside Drive) New York, NY 10...",1.716,"{'address': 'West 72nd St', 'crossStreet': 'Ri...",40.781111,-73.986382,"{'type': 'Point', 'coordinates': [-73.98638169..."
3,dog hairdresser,"87th St. & Riverside Dr. (87th St) New York, N...",1.410,"{'address': '87th St. & Riverside Dr.', 'cross...",40.790392,-73.980230,"{'type': 'Point', 'coordinates': [-73.98023043..."
4,dog hairdresser,"59th St (6th Ave) New York, NY 10019 United St...",2.085,"{'address': '59th St', 'crossStreet': '6th Ave...",40.766147,-73.975803,"{'type': 'Point', 'coordinates': [-73.97580304..."
5,dog hairdresser,"700 W 44th St. (at 12th Ave.) New York, NY 100...",3.632,"{'address': '700 W 44th St.', 'crossStreet': '...",40.763595,-74.000465,"{'type': 'Point', 'coordinates': [-74.00046544..."
6,dog hairdresser,W 81st St. (btwn Columbus & Central Park West)...,0.645,"{'address': 'W 81st St.', 'crossStreet': 'btwn...",40.782223,-73.973742,"{'type': 'Point', 'coordinates': [-73.97374214..."
7,dog hairdresser,Morningside Ave (btw 114th St & 119th St) New ...,2.410,"{'address': 'Morningside Ave', 'crossStreet': ...",40.804403,-73.959159,"{'type': 'Point', 'coordinates': [-73.95915933..."
8,dog hairdresser,"51st Avenue (Center Blvd) Long Island City, NY...",4.494,"{'address': '51st Avenue', 'crossStreet': 'Cen...",40.743340,-73.959869,"{'type': 'Point', 'coordinates': [-73.95986854..."
9,dog hairdresser,"E 63rd St (at FDR Dr) New York, NY 10021 Unite...",2.704,"{'address': 'E 63rd St', 'crossStreet': 'at FD...",40.760183,-73.956919,"{'type': 'Point', 'coordinates': [-73.95691858..."


In [102]:
str_air_vegan_nba_dog_offices=pd.DataFrame(wtematch(datany))

In [103]:
str_air_vegan_nba_dog_offices.drop_duplicates(subset='name', keep='first')

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geopoint
0,RayV,games_video,None,"150 West 77, Suite 4",None,None,New York,NY,USA,40.781159,-73.978332,"{'type': 'Point', 'coordinates': [-73.978332, ..."
